<a href="https://colab.research.google.com/github/Noodle-bg/Baby-GPT/blob/main/Baby_GPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
with open('Friends_script.txt','r',encoding='utf-8') as f:
  text = f.read()

In [2]:
print(len(text))

4176958


In [3]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)

	
 !"#$%&'()*+,-./0123456789:;<=>?@ABCDEFGHIJKLMNOPQRSTUVWXYZ[]_`abcdefghijklmnopqrstuvwxyz{|}¦­Éâäçèéíī–—‘’“”…€™�
114


In [4]:
stoi = {ch:i for i,ch in enumerate(chars)}
itos = {i:ch for i,ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s]
decode = lambda l:''.join([itos[i] for i in l])


print(encode("Hello world"))
print(decode(encode("Hello world")))

[42, 69, 76, 76, 79, 2, 87, 79, 82, 76, 68]
Hello world


In [5]:
import torch
data = torch.tensor(encode(text),dtype=torch.long)
print(data.shape,data.dtype)
print(data[:1000])

torch.Size([4176958]) torch.int64
tensor([47, 49, 48, 43, 37, 35, 28,  2,  2, 54, 72, 69, 82, 69,  9, 83,  2, 78,
        79, 84, 72, 73, 78, 71,  2, 84, 79,  2, 84, 69, 76, 76,  3,  2, 42, 69,
         9, 83,  2, 74, 85, 83, 84,  2, 83, 79, 77, 69,  2, 71, 85, 89,  2, 43,
         2, 87, 79, 82, 75,  2, 87, 73, 84, 72,  3,  1, 44, 49, 39, 59, 28,  2,
         2, 37,  9, 77, 79, 78, 14,  2, 89, 79, 85,  9, 82, 69,  2, 71, 79, 73,
        78, 71,  2, 79, 85, 84,  2, 87, 73, 84, 72,  2, 84, 72, 69,  2, 71, 85,
        89,  3,  2, 54, 72, 69, 82, 69,  9, 83,  2, 71, 79, 84, 84, 65,  2, 66,
        69,  2, 83, 79, 77, 69, 84, 72, 73, 78, 71,  2, 87, 82, 79, 78, 71,  2,
        87, 73, 84, 72,  2, 72, 73, 77,  3,  1, 37, 42, 35, 48, 38, 46, 39, 52,
        28,  2,  2, 53, 79,  2, 68, 79, 69, 83,  2, 72, 69,  2, 72, 65, 86, 69,
         2, 65,  2, 72, 85, 77, 80, 33,  2, 35,  2, 72, 85, 77, 80,  2, 65, 78,
        68,  2, 65,  2, 72, 65, 73, 82, 80, 73, 69, 67, 69, 33,  1, 50, 42, 49,
      

In [7]:
n = int(0.9*len(data))
train_data=data[:n]
val_data = data[n:]

In [10]:
torch.manual_seed(42)
batch_size = 4 # How much parallel computation happens at a time
block_size = 8 # Context length

def get_batch(split):
  data = train_data if split == 'train' else val_data
  ix = torch.randint(len(data)-block_size,(batch_size,))
  x = torch.stack([data[i:i+block_size]for i in ix])
  y= torch.stack([data[i+1:i+block_size+1]for i in ix])
  return x,y


In [11]:
xb,yb = get_batch('train')
print(xb)

tensor([[89,  2, 65, 82, 69,  2, 71, 79],
        [ 2,  2, 47, 69, 33,  2, 43,  9],
        [83,  2, 71, 79,  2, 84, 79,  2],
        [54, 79,  2, 52, 65, 67, 72, 69]])


In [15]:
import torch
import torch.nn as nn
from torch.nn import functional as F

torch.manual_seed(42)

class BigramLanguageModel(nn.Module):
  def __init__(self,vocab_size):
    super().__init__()
    self.token_embedding_table = nn.Embedding(vocab_size,vocab_size)


  def forward(self,idx,targets=None):
    logits = self.token_embedding_table(idx)
    if targets == None:
      loss = None
    else:
      B, T,C = logits.shape
      logits = logits.view(B*T,C)
      targets = targets.view(B*T)
      loss = F.cross_entropy(logits,targets)
    return logits,loss



  def generate(self,idx,max_new_tokens):
    for _ in range(max_new_tokens):
      logits,loss = self(idx)
      # Focus only on the last time step
      logits = logits[:,-1,:] # Becomes (B,C)
      probs = F.softmax(logits,dim = -1) # (B,C)
      idx_next = torch.multinomial(probs,num_samples = 1) # (B,1)
      idx = torch.cat((idx,idx_next),dim = 1) #(B,T+1)
    return idx


m = BigramLanguageModel(vocab_size)
logits , loss = m(xb,yb)
print(logits.shape)
print(loss)


print(decode(m.generate(torch.zeros((1,1),dtype = torch.long),max_new_tokens = 100)[0].tolist()))

torch.Size([32, 114])
tensor(5.0856, grad_fn=<NllLossBackward0>)
	.>­¦IYyY­|¦īw2”VK5P;JNBfNkA`OIWn|â)=1‘	éè1[wíä`rxoéèT?­Tj5Lä<	­!:’yI¦R’y W?i€€[èī™JMg”H�q5!1D.ävéçtl


In [16]:
optimizer = torch.optim.AdamW(m.parameters(),lr=1e-3)

In [22]:
batch_size = 32
for steps in range(10000):
  xb,yb = get_batch('train')
  logits , loss= m(xb,yb)
  optimizer.zero_grad(set_to_none = True)
  loss.backward()
  optimizer.step()
print(loss.item())

2.3554248809814453


In [23]:
print(decode(m.generate(torch.zeros((1,1),dtype = torch.long),max_new_tokens = 100)[0].tolist()))

	(ly! h!!
ROoish! V(le ce g HUNICHE:    no see ICahe NOknd dou i, GEBEYovo   s.
JOhes ti, dere sthahe
